In [1]:
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "DataFrames",
    "uCSV"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


In [2]:
project_dir = dirname(pwd())
data_dir = mkpath(joinpath(project_dir, "data"))

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data"

In [13]:
viral_reference_fastas_dir = mkpath(joinpath(data_dir, "viral-reference-fastas"))
reference_fastas = sort(filter(x -> occursin(r"\.f(na|asta)\.gz$", x), readdir(viral_reference_fastas_dir, join=true)), by=x->filesize(x))

5-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/GCF_000001405.40_GRCh38.p14_genomic.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/ictv-examplar-viruses.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/ref_viruses_rep_genomes.fasta.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/IMGVR_all_nucleotides-high_confidence.fna.gz"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/nt_viruses.fasta.gz"

In [4]:
slurm_logdir = mkpath("$(homedir())/workspace/slurmlogs")

"/home/cjprybol/workspace/slurmlogs"

In [5]:
# samples = readdir(joinpath(dirname(project_dir), "viral-exposome-discovery", "data", "SRA"), join=true)

In [6]:
exposome_metadata_dir = joinpath(project_dir, "metadata")

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/metadata"

In [7]:
exposome_environmental_data = DataFrames.DataFrame(uCSV.read(
    joinpath(exposome_metadata_dir, "metadata_exposome.rds.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

joint_sample_metadata = DataFrames.DataFrame(uCSV.read(
    joinpath(exposome_metadata_dir, "exposome/joint_sample_metadata.tsv"),
    delim='\t',
    header=1,
    typedetectrows=300
))

@assert joint_sample_metadata[!, "Library Name"] == joint_sample_metadata[!, "LibraryName"]

joint_metadata = DataFrames.innerjoin(
    joint_sample_metadata,
    exposome_environmental_data,
    on="Library Name" => "samplenames")

sample_directories = sort(joinpath.(project_dir, "data", "SRA", joint_metadata[!, "Run"]))

┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213
┌ Warning: Large values for `typedetectrows` will reduce performance. Consider manually declaring the types of columns using the `types` argument instead.
└ @ uCSV ~/.julia/packages/uCSV/goLRH/src/read.jl:213


213-element Vector{String}:
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399459"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399460"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399461"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399462"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399464"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399485"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399486"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399487"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399488"
 "/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/SRA/SRR6399490"
 "/oak/stanford/scg/la

In [14]:
# 64 ictv-examplar-viruses.fna.gz
# 64 ref_viruses_rep_genomes.fasta.gz
# fasta_file = reference_fastas[2]
# fasta_file = reference_fastas[3]
memory = 64
threads = 8

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/ref_viruses_rep_genomes.fasta.gz"

In [17]:
fasta_file = reference_fastas[1]

"/oak/stanford/scg/lab_mpsnyder/cjprybol/Mycelia/projects/viral-exposome/data/viral-reference-fastas/GCF_000001405.40_GRCh38.p14_genomic.fna.gz"

In [18]:
memory = 128
threads = 16
# 128 GCF_000001405.40_GRCh38.p14_genomic.fna.gz

16

In [19]:
for sample in sample_directories
    forward = joinpath(sample, "trim_galore", "$(basename(sample))_1_val_1.fq.gz")
    reverse = joinpath(sample, "trim_galore", "$(basename(sample))_2_val_2.fq.gz")
    outdir = mkpath(joinpath(sample, "alignments"))
    outfile = joinpath(outdir, basename(fasta_file) * ".sorted.bam")
    # @show outfile
    if !isfile(outfile) || (filesize(outfile) == 0)
        cmd =
        """
        $(Mycelia.MAMBA) run --live-stream -n bwa-mem2 bwa-mem2 mem -t $(threads) $(fasta_file) $(forward) $(reverse) \\
        | $(Mycelia.MAMBA) run --live-stream -n samtools samtools sort -u --threads $(threads) \\
        | $(Mycelia.MAMBA) run --live-stream -n samtools samtools view --bam --with-header --output $(outfile)
        """
        Mycelia.sbatch(
            job_name = "bwa-mem2.$(basename(fasta_file)).$(basename(sample))",
            mail_user = "cameron.prybol@gmail.com",
            logdir = slurm_logdir,
            partition = "batch",
            account = "mpsnyder",
            mem_gb = memory,
            cpus_per_task= threads,
            cmd = cmd)
    else
        @info "$(outfile) already present..."
    end
end

Submitted batch job 42474709
Submitted batch job 42474710
Submitted batch job 42474711
Submitted batch job 42474712
Submitted batch job 42474713
Submitted batch job 42474714
Submitted batch job 42474715
Submitted batch job 42474716
Submitted batch job 42474717
Submitted batch job 42474718
Submitted batch job 42474719
Submitted batch job 42474720
Submitted batch job 42474721
Submitted batch job 42474722
Submitted batch job 42474723
Submitted batch job 42474724
Submitted batch job 42474725
Submitted batch job 42474726
Submitted batch job 42474727
Submitted batch job 42474728
Submitted batch job 42474729
Submitted batch job 42474730
Submitted batch job 42474731
Submitted batch job 42474732
Submitted batch job 42474733
Submitted batch job 42474734
Submitted batch job 42474735
Submitted batch job 42474736
Submitted batch job 42474737
Submitted batch job 42474738
Submitted batch job 42474739
Submitted batch job 42474740
Submitted batch job 42474741
Submitted batch job 42474742
Submitted batc